In [1]:
import gym
import numpy as np
from math import sqrt
import time
import random

In [2]:
def get_discrete_obs(observation):
    t = (observation* scale).astype(int)
    return tuple(t)

In [3]:
NEG_INF = -99
env = gym.make('CartPole-v1')
scale = 12


In [4]:

def set_state_action(sa):
    global state_action
    if sa not in state_action:
        state_action[sa] = 0

def get_action(obs):
    global env
    
    t = random.random()
    if t < exploration:
        return env.action_space.sample()
    else:
        act = env.action_space.sample()
        val = 0
        for action in range(env.action_space.n): 
            sa = (obs,action)
            set_state_action(sa)
            if state_action[sa] > val:
                act = action
                val = state_action[sa]
            
        return act


## This is SARSA
def update_state_action(n,states,actions,rewards):
    current_sa = (states[0],actions[0])
    set_state_action(current_sa)
    
    total_rewards = 0
    for i in range(len(rewards)):
        total_rewards += pow(discount_rate,i+1) * rewards[i]
    
    next_action = get_action(states[-1])
    next_sa = (states[-1],next_action)  
    set_state_action(next_sa)

    
#     print(states,actions,rewards,state_action[current_sa])
#     print(current_sa,total_rewards + pow(discount_rate,n)*state_action[next_sa],state_action[current_sa])
    
    state_action[current_sa] = state_action[current_sa] + learning_rate*(total_rewards + pow(discount_rate,n)*state_action[next_sa] - state_action[current_sa]) 
#     print(state_action[current_sa])


    

In [8]:
def train(n,no_episodes = 10):
    global exploration,env
    avg_len = 0
    
    for ep in range(no_episodes):
        states = []
        actions = []
        rewards = []
        
        obs = get_discrete_obs(env.reset())
        
        ep_len = 0
        while True:
            ep_len += 1
            action = get_action(obs)
            new_obs, reward, done, _ = env.step(action)
            new_obs = get_discrete_obs(new_obs)
            
            states.append(obs)
            actions.append(action)
            rewards.append(reward)
            obs = new_obs
            if len(states) > n:
                states = states[1:]
                actions = actions[1:]
                rewards = rewards[1:]
            
            if done:
                break
                
            if len(states) < n:
                continue
            
                
            update_state_action(n,states,actions,rewards)
            
            if done:
                break
        
        exploration = exploration * 0.999
        avg_len += ep_len
        
        if ep % 200 == 0:
            print(avg_len/200)
            avg_len = 0

In [9]:
## (state,action) -> reward
state_action = {}

exploration = 0.7
learning_rate = 1
discount_rate = 1
n = 12

In [10]:
train(n,5000)

0.115
23.07
25.35
20.505
24.61
26.665
22.72
25.44
23.67
24.09
21.11
20.72
25.04
19.515
19.185
21.815
15.645
22.14
22.6
24.17
19.29
21.86
21.365
26.0
21.895
